In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import numpy as np
import pandas as pd
import torch
import gc
import syft as sy
from syft.core.adp.entity import Entity
from pympler.asizeof import asizeof #pip install pympler
from syft import serialize
from syft import deserialize
from typing import Tuple, Dict, List
from pathlib import Path

import timeit
from syft.core.tensor.autodp.dp_tensor_converter import convert_to_gamma_tensor

from functools import reduce

from syft.util import download_file
from syft.util import get_root_data_path
from syft.core.adp.adversarial_accountant import AdversarialAccountant
from nacl.signing import VerifyKey, SigningKey

In [2]:
def download_spicy_bird_benchmark() -> Tuple[Dict[str, Path], List[str]]:
    file_suffix = "_rows_dataset_sample.parquet"
    BASE_URL = "https://raw.githubusercontent.com/madhavajay/datasets/main/spicy_bird/"
    sizes = ["100K", "250K", "500K", "750K", "1M"]
    folder_name = "spicy_bird"
    dataset_path = get_root_data_path() / folder_name
    paths = []
    for size in sizes:
        filename = f"{size}{file_suffix}"
        url = f"{BASE_URL}{filename}"
        print(url)
        path = download_file(url=url, full_path=dataset_path / filename)
        paths.append(path)
    return dict(zip(sizes, paths)), sizes

In [3]:
files, ordered_sizes = download_spicy_bird_benchmark()

https://raw.githubusercontent.com/madhavajay/datasets/main/spicy_bird/100K_rows_dataset_sample.parquet
https://raw.githubusercontent.com/madhavajay/datasets/main/spicy_bird/250K_rows_dataset_sample.parquet
https://raw.githubusercontent.com/madhavajay/datasets/main/spicy_bird/500K_rows_dataset_sample.parquet
https://raw.githubusercontent.com/madhavajay/datasets/main/spicy_bird/750K_rows_dataset_sample.parquet
https://raw.githubusercontent.com/madhavajay/datasets/main/spicy_bird/1M_rows_dataset_sample.parquet


In [4]:
files

{'100K': PosixPath('/Users/madhavajay/.syft/data/spicy_bird/100K_rows_dataset_sample.parquet'),
 '250K': PosixPath('/Users/madhavajay/.syft/data/spicy_bird/250K_rows_dataset_sample.parquet'),
 '500K': PosixPath('/Users/madhavajay/.syft/data/spicy_bird/500K_rows_dataset_sample.parquet'),
 '750K': PosixPath('/Users/madhavajay/.syft/data/spicy_bird/750K_rows_dataset_sample.parquet'),
 '1M': PosixPath('/Users/madhavajay/.syft/data/spicy_bird/1M_rows_dataset_sample.parquet')}

In [5]:
df = pd.read_parquet(files["1M"])
print("Number of Rows: ",df.shape[0])
df.head()
df = df[:2]
print(df.shape[0])

Number of Rows:  1000000
2


In [6]:
name = "Tweets- 100000 rows dataset "
impressions = ((np.array(list(df['impressions'])))).astype(np.int32)
publication_title = ((list(df['user_id'])))

entities = list()
for i in range(len(publication_title)):
    entities.append(Entity(name=str(publication_title[i])))
    
tweets_data = sy.Tensor(impressions).private(min_val=0, max_val=30, entities = entities)

In [7]:
%%time
left_result = tweets_data.sum()

CPU times: user 327 µs, sys: 71 µs, total: 398 µs
Wall time: 367 µs


In [8]:
key = SigningKey.generate()
the_actual_key = key.verify_key

In [9]:
type(left_result.child)

syft.core.tensor.autodp.intermediate_gamma.IntermediateGammaTensor

In [10]:
type(left_result)

syft.core.tensor.tensor.Tensor

In [11]:
acc = AdversarialAccountant()
left_result.publish(sigma=1000, acc=acc, user_key=the_actual_key)

creating accountant with a dictionary
are we using this publish? <syft.core.adp.adversarial_accountant.AdversarialAccountant object at 0x152954a60> 1000 b'\xbe\xb8-\x04\xf3\xf8\x12\xb7\x18JT\xd4\xac5T\xf6\xc56\xcfc\xd7n\x06\x1a\x16T\xd29-\x90]\xd0'
running flat scalars
running flat scalars 2
running flat scalars 3 0
running flat scalars 4 0
running flat scalars 4 1
finished running flat scalars
executing publish with <class 'list'> <class 'syft.core.adp.adversarial_accountant.AdversarialAccountant'> <class 'nacl.signing.VerifyKey'> <class 'int'> <class 'bool'>
get_all_entity_mechanisms <class 'list'> <class 'int'> <class 'bool'>
for s in scalars
for entity in entities
for _, _ in get_mechanism_for_entity
if flat_entity not in entity_to_mechanisms
for entity in entities
for _, _ in get_mechanism_for_entity
if flat_entity not in entity_to_mechanisms
finished getting all entity mechanisms <class 'dict'>
calling temp append <class 'dict'>
finished temp_append


Has Budget:True
YOU'VE SPENT

/Users/madhavajay/dev/PySyft/.tox/syft.jupyter/lib/python3.9/site-packages/scipy/optimize/optimize.py:2621: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


array([17346.44276525])